# GoogLeNet en base a un algoritmo usado para mnist

Link oritiginal: https://github.com/melty-chocolate/GoogLeNet-Inception-TensorFlow
migrado a TF 2.0

este algoritmo de googlenet no me funciono.
https://github.com/malikmdk/GoogLeNet-with-Tensorflow-2.0

In [1]:
#prueba de gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout,Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
import pydot
import pickle

In [3]:
# se cargan X e y desde los datos preprocesados
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

X = X/255

print( X.shape , y.shape)




(4259, 229, 229, 3) (4259, 7)


In [4]:
def Gnet_model(input_shape, classes):
    
    X_input = Input(input_shape)
  
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (5, 5), name = 'conv1',padding = 'same')(X_input)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    # MAXPOOL
    X = MaxPooling2D((2, 2),strides = (2, 2), name = 'max_pool1')(X)
    
    # branch1
    # CONV 1X1
    b1_X = Conv2D(64, (1, 1), name = 'conv2',padding = 'valid')(X)
    b1_X = BatchNormalization(axis = 3, name = 'bn2')(b1_X)
    b1_X = Activation('relu')(b1_X)
    b1_X = MaxPooling2D((3, 3), name='max_pool2')(b1_X)

    # branch2
    # CONV 1x1 -> CONV 3x3
    b2_X = Conv2D(64, (1, 1), name = 'conv3',padding = 'same')(X)
    b2_X = BatchNormalization(axis = 3, name = 'bn3')(b2_X)
    b2_X = Activation('relu')(b2_X)
    b2_X = Conv2D(64, (3, 3), name = 'conv4',padding = 'same')(b2_X)
    b2_X = BatchNormalization(axis = 3, name = 'bn4')(b2_X)
    b2_X = Activation('relu')(b2_X)
    b2_X = MaxPooling2D((3, 3), name='max_pool3')(b2_X)
    
    # branch3
    # CONV 1x1 -> CONV 5x5
    b3_X = Conv2D(64, (1, 1), name = 'conv5',padding = 'same')(X)
    b3_X = BatchNormalization(axis = 3, name = 'bn5')(b3_X)
    b3_X = Activation('relu')(b3_X)
    b3_X = Conv2D(64, (5, 5), name = 'conv6',padding = 'same')(b3_X)
    b3_X = BatchNormalization(axis = 3, name = 'bn6')(b3_X)
    b3_X = Activation('relu')(b3_X)
    b3_X = MaxPooling2D((3, 3), name='max_pool4')(b3_X)
    
    # branch4
    # MAXPOOL 3x3 -> CONV 1x1
    b4_X = MaxPooling2D((3, 3), name='max_pool5')(X)
    b4_X = Conv2D(64, (1, 1), name = 'conv7',padding = 'same')(b4_X)
    b4_X = BatchNormalization(axis = 3, name = 'bn7')(b4_X)
    b4_X = Activation('relu')(b4_X)
    
    # CONCAT
    concat = tf.keras.layers.concatenate([b1_X, b2_X, b3_X, b4_X],axis=3)
    X = Activation('relu')(concat)
    
    #X = Flatten()(X)
    X = Dense(128, activation='relu', name='fc1')(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    # Create model. 
    model = Model(inputs = X_input, outputs = X, name='MNIST')

    return model



In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

modelo = Gnet_model(input_shape=(229,229,3), classes = 7)
modelo.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

modelo.summary()

Model: "MNIST"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 229, 229, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 229, 229, 32) 2432        input_1[0][0]                    
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 229, 229, 32) 128         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 229, 229, 32) 0           bn1[0][0]                        
______________________________________________________________________________________________

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [7]:
modelo.fit(X_train, y_train, epochs = 20, batch_size = 64 )

Epoch 1/20
54/54 [==============================] - 12s 221ms/step - loss: 4.4709 - accuracy: 0.6008
Epoch 2/20
54/54 [==============================] - 11s 208ms/step - loss: 0.3234 - accuracy: 0.8873
Epoch 3/20
54/54 [==============================] - 11s 208ms/step - loss: 0.1648 - accuracy: 0.9436
Epoch 4/20
54/54 [==============================] - 11s 208ms/step - loss: 0.1205 - accuracy: 0.9615
Epoch 5/20
54/54 [==============================] - 11s 208ms/step - loss: 0.0727 - accuracy: 0.9736
Epoch 6/20
54/54 [==============================] - 11s 208ms/step - loss: 0.0539 - accuracy: 0.9839
Epoch 7/20
54/54 [==============================] - 11s 208ms/step - loss: 0.0419 - accuracy: 0.9862
Epoch 8/20
54/54 [==============================] - 11s 208ms/step - loss: 0.0153 - accuracy: 0.9956
Epoch 9/20
54/54 [==============================] - 11s 207ms/step - loss: 0.0170 - accuracy: 0.9944
Epoch 10/20
54/54 [==============================] - 11s 208ms/step - loss: 0.0189 - accura

In [8]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = modelo.evaluate(X_test, y_test, batch_size=128)
print('test loss, test acc:', results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = modelo.predict(X_test[:3])
print('predictions shape:', predictions.shape)
print('predicciones: ' , predictions[0],predictions[1],predictions[2])


# Evaluate on test data
7/7 [==============================] - 2s 229ms/step - loss: 0.3041 - accuracy: 0.9425
test loss, test acc: [0.3041243255138397, 0.9424882531166077]

# Generate predictions for 3 samples
predictions shape: (3, 7)
predicciones:  [4.9357592e-15 6.6917676e-14 6.1487108e-06 1.9291150e-07 2.9127109e-12
 9.9999094e-01 2.7856443e-06] [1.2681151e-09 8.2826638e-01 5.8903615e-09 2.9381201e-02 1.4235014e-01
 7.9741893e-09 2.2676832e-06] [8.9523483e-18 9.9855922e-12 7.6378628e-14 4.1658180e-18 8.1310233e-17
 8.3465435e-18 1.0000000e+00]
